### Wikipedia Web Scraping, Data Analysis and Machine Learning

This project involves using Beautiful Soup to extract data from Wikipedia's list of top 50 largest companies by revenue.

https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue

Following successful web scraping and formatting into a DataFrame, exploratory data analysis is performed to better understand the distributions and relatinships in the data. 

Increasingly more complex classification models are then built to classify whether a company is state-owned or not.

### Part 1: web scraping, data formatting and cleaning

#### Web scraping and column identification

In [ ]:
# Import necessary libraries
from bs4 import BeautifulSoup
import requests

In [ ]:
# Define the URL
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue'

# Use the requests library to send a GET request to the URL to retrieve the HTML content
page = requests.get(url)

# Parse the HTML with Beautiful Soup to create a soup object
soup = BeautifulSoup(page.text, 'html.parser')

# Find all tables on the wikipedia page and select the first one
table = soup.find_all('table')[0]

In [ ]:
# Find the header row of the specified table and iterate over each table header (<th>) element 
# within the header row and extract the text using the get_text() method. Strip whitespace from 
# the text and append the cleaned text to the column_names list.
header_row = table.find('tr')
column_names = []

for header in header_row.find_all('th'):
    header_text = header.get_text(strip=True)
    column_names.append(header_text)

#### Row and data identification

In [ ]:
data_rows = []

# Iterate over each row (<tr>), starting from the third row (index 2) to skip the header rows.
for row in table.find_all('tr')[2:]:
    data_row = [] # Within each iteration of the outer loop,initialise an empty list to store the data for the current row.
    for cell in row.find_all(['td', 'th']):
        cell_text = cell.get_text(strip=True)
        # Check if the cell contains an image, and check whether the alt text for the image is 'Yes' or 'No'
        img = cell.find('img')
        if img:
            if img['alt'] == 'Yes': 
                cell_text = 'Yes'
            elif img['alt'] == 'No':
                cell_text = 'No'
        if cell_text != 'USD millions':
            data_row.append(cell_text)
    data_rows.append(data_row)

#### DataFrame creation

In [ ]:
# Create a pandas DataFrame and add the columns and data rows.

import pandas as pd

# Create DataFrame
df = pd.DataFrame(data_rows, columns=column_names)

# Display DataFrame
df.tail()

#### DataFrame formatting and cleaning

In [ ]:
# Some formatting and cleaning is required here: the 'Ref.' column is not needed and the 'Headquarters[note 1]' column is
# renamed for clarity. The 'Revenue' and 'Profit' columns need to be formatted to remove the '$' so they can be converted to
# integers. Commas in columns containing numbers are also removed to make analysis easier.


# Drop the 'Ref.' column
df.drop(columns=['Ref.'], inplace=True)


# Rename the 'Headquarters[note 1]' column to 'Headquarters'
df.rename(columns={'Headquarters[note 1]': 'Headquarters'}, inplace=True)


# Remove leading '$' from 'Revenue', 'Profit', and 'Revenue per worker' columns
df['Revenue'] = df['Revenue'].str.replace('$', '', regex=False)
df['Profit'] = df['Profit'].str.replace('$', '', regex=False)
df['Revenue per worker'] = df['Revenue per worker'].str.replace('$', '', regex=False)


# Remove commas from numeric columns
df['Profit'] = df['Profit'].str.replace(',', '')
df['Revenue'] = df['Revenue'].str.replace(',', '')
df['Employees'] = df['Employees'].str.replace(',', '')
df['Revenue per worker'] = df['Revenue per worker'].str.replace(',', '')


# Convert the 'Profit' column to integers, skipping non-convertible values
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce').astype('Int64')


# Convert the 'Revenue' column to integers
df['Revenue'] = df['Revenue'].astype(int)


# Convert the 'Employees' column to integers
df['Employees'] = df['Employees'].astype(int)


# Convert the 'Revenue per worker' column to integers
df['Revenue per worker'] = df['Revenue per worker'].astype(float)

# Rename columns to specify units
df.rename(columns={'Revenue': 'Revenue (USD millions)', 'Profit': 'Profit (USD millions)', 'Revenue per worker': 'Revenue per worker (USD)'}, inplace=True)


In [ ]:
# Save DataFrame to CSV with explicit encoding
df.to_csv('companies_data.csv', index=False)

### Part 2: exploratory data analysis 

#### Histogram creation to visualise distribution of numeric columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select numerical columns
numerical_columns = df.select_dtypes(include=['int32', 'Int64', 'float64'])

# Set up the figure and axes
fig, axes = plt.subplots(nrows=len(numerical_columns.columns), ncols=1, figsize=(10, 5*len(numerical_columns.columns)))

# Loop through each numerical column
for i, col in enumerate(numerical_columns.columns):
    # Histogram
    sns.histplot(data=df, x=col, kde=True, ax=axes[i])
    axes[i].set_title(f'Histogram of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


#### Scatter plot of revenue and profit

In [ ]:
# Filter out rows with NA revenue
filtered_df = df.dropna(subset=['Profit (USD millions)'])

plt.figure(figsize=(10, 6))
plt.scatter(filtered_df['Revenue (USD millions)'], filtered_df['Profit (USD millions)'], alpha=0.5)
plt.title('Scatter Plot of Revenue vs Profit')
plt.xlabel('Revenue (USD millions)')
plt.ylabel('Profit (USD millions)')
plt.grid(True)
plt.show()


#### Calculate the pearson correlation coefficient, ignoring NA rows

In [ ]:
from scipy.stats import pearsonr

# Filter the DataFrame to include only the columns of interest
filtered_df = df[['Revenue (USD millions)', 'Profit (USD millions)']].copy()

# Drop rows with any NaN values in the selected columns
filtered_df.dropna(subset=['Revenue (USD millions)', 'Profit (USD millions)'], inplace=True)

# Calculate the correlation coefficient using pearsonr
correlation, _ = pearsonr(filtered_df['Revenue (USD millions)'], filtered_df['Profit (USD millions)'])
print("Correlation coefficient:", correlation)

For companies with lower profit and revenue, it seems there is a stronger correlation between these variables.

In [ ]:
# Filter the DataFrame to include only rows where revenue is 200000 or lower
low_revenue_df = df[df['Revenue (USD millions)'] <= 220000].copy()

# Drop rows with missing values
low_revenue_df.dropna(subset=['Revenue (USD millions)', 'Profit (USD millions)'], inplace=True)

# Calculate the correlation coefficient for this subset
correlation_low_revenue, _ = pearsonr(low_revenue_df['Revenue (USD millions)'], low_revenue_df['Profit (USD millions)'])
print("Correlation coefficient for companies with revenue of 220000 million USD or lower:", correlation_low_revenue)

In [ ]:
# Print list of companies with a 'low' revenue

print(low_revenue_df[['Name', 'Revenue (USD millions)', 'Profit (USD millions)']].to_string(index=False))

#### Checking the number of state-owned vs privately-owned companies

In [ ]:
state_owned_counts = df['State-owned'].value_counts()
print("Number of state-owned companies:", state_owned_counts['Yes'])
print("Number of private companies:", state_owned_counts['No'])

### Part 3: Classification models

#### Logistic regression

I want to see whether a model can accurately predict whether a company is state-owned based on its numeric features. We can see that there are over double the number of private companies to state-owned, so when splitting the data into training and test sets, the split is stratified to keep the proportion of state-owned companies the same in each set. This eliminates sample bias and reflects the population being studied. 

Logistic regression is first used to classify the data into state-owned or not state-owned.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Drop rows where profit is NaN
df.dropna(subset=['Profit (USD millions)'], inplace=True)

# Assume X contains your features and y contains the target variable (state-owned or not)
X = df[['Revenue (USD millions)', 'Profit (USD millions)', 'Employees', 'Revenue per worker (USD)']]
y = df['State-owned']

# Split the data into training and testing sets, stratifying by the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Display the proportions of state-owned and private companies in the training set
print("Training set:")
print(y_train.value_counts(normalize=True))

# Display the proportions of state-owned and private companies in the testing set
print("\nTesting set:")
print(y_test.value_counts(normalize=True))

# Create and train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
predictions = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, predictions)
print("\nAccuracy:", accuracy)


These poor results show that just guessing a company is not state-owned would be better, as most of the companies are not state-owned. A more balanced dataset may produce better results. 

Furthermore, the features used for classification may not be important or the relationships may not be linear. Logistic regression assumes linear relationships between the input variables and the target variable; if these relationships are not linear then logistic regression would naturally perform poorly.

#### Random forest

Random forest will now be used for classification because it is capable of capturing non-linear relationships between features in data. This may lead to improved performance from logistic regression if the relationhips are non-linear.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train the Random Forest Classifier model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the testing set
rf_predictions = rf_model.predict(X_test)

# Evaluate the model's accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Classifier Accuracy:", rf_accuracy)

These improved results suggest the the relationships may be non-linear, although the model still performs poorly.

#### XGBoost

The final classification model used is XGBoost. This is a powerful gradient-boosting algorithm which has gained popularity for being efficient and accurate. Here a grid search of hyperparameter values is performed to find the best combination from a defined list. An XGBoost model is trained and tested using these values, and the performance is evaluated.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Define parameters for the XGBoost model
xgb_params = {
    'objective': 'binary:logistic',  
    'eval_metric': 'error',           
}

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Define the dataset
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)

# Define the parameter grid for the grid search
param_grid = {
    'learning_rate': [0.3, 0.6],       
    'max_depth': [4, 6],                 
    'min_child_weight': [1, 3],          
    'subsample': [0.8, 1.0],                
    'n_estimators': [50, 100],
    'gamma': [0, 0.2],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1],
}

# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier(**xgb_params)

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train_encoded)

# Get the best parameters from the grid search
best_params_xgb = grid_search.best_params_
print("Best Parameters for XGBoost:", best_params_xgb)

# Train the XGBoost model with the best parameters
best_xgb_model = xgb.XGBClassifier(**xgb_params, **best_params_xgb)
best_xgb_model.fit(X_train, y_train_encoded)

# Make predictions on the test set
y_pred_test_xgb = best_xgb_model.predict(X_test)

# Decode the predictions
y_pred_test_decoded = label_encoder.inverse_transform(y_pred_test_xgb)

# Calculate accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_test_decoded)
print("Test Accuracy (XGBoost):", accuracy_xgb)


These results are improved over random forest, but still not very good. It is encouraging to see successive models getting better results, but accuracy may be limited due to a number of reasons.

. There may be more influential features which are not included in the model, in the future it may be possible to add extra information for these companies.

. The hyperparameter set may not be optimal; while grid search was used to find the best combination, there are many more adjustable hyperparameters and values to test.

. The dataset is small. In the future, a much larger dataset with thousands or more companies can be used, which may improve results for all models tested.

. There are companies which have an NA value for profit. I decided to remove these from the model, but replacing them with something like the mean profit of the other companies, or just '0' would increase the size of the dataset and potentially improve results.